In [1]:
import os

In [2]:
%pwd

'/Users/kunal/Developer/DL-Project/End-to-End-Kidney-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/kunal/Developer/DL-Project/End-to-End-Kidney-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
import numpy as np
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
"""
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    # load the model from artifacts
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    
    # split the data into train and test data
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20 # 20% data for validation
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    # save the model
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    # training method
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )
        

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
"""

In [11]:
# Run on CPU (for Mac M1/M2 compatibility)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Force eager execution
tf.compat.v1.enable_eager_execution()
print("Eager execution enabled:", tf.executing_eagerly())

class Training:
    def __init__(self, config):
        self.config = config

    def get_base_model(self):
        # Load the model and compile it
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"]
        )
        print("Model loaded and compiled.")

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20  # 20% for validation
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
            class_mode="sparse"  # Use sparse labels (integers)
        )

        # Validation generator
        valid_datagenerator = ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # Training generator with optional augmentation
        if self.config.params_is_augmentation:
            train_datagenerator = ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # Convert the generators to tf.data.Datasets.
        # We use a generator expression to yield (x, y) batches.
        self.train_dataset = tf.data.Dataset.from_generator(
            lambda: ((x, y) for x, y in self.train_generator),
            output_signature=(
                tf.TensorSpec(shape=(None, *self.config.params_image_size), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )
        self.valid_dataset = tf.data.Dataset.from_generator(
            lambda: ((x, y) for x, y in self.valid_generator),
            output_signature=(
                tf.TensorSpec(shape=(None, *self.config.params_image_size), dtype=tf.float32),
                tf.TensorSpec(shape=(None,), dtype=tf.int32)
            )
        )

        # Map over the dataset to ensure any RaggedTensors are converted to dense.
        def to_dense(x, y):
            if isinstance(x, tf.RaggedTensor):
                x = x.to_tensor()
            return x, y

        self.train_dataset = self.train_dataset.map(to_dense)
        self.valid_dataset = self.valid_dataset.map(to_dense)

        print("Data generators converted to tf.data.Datasets.")

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        print(f"Model saved at {path}")

    def train(self):
        # Compute steps from the original generator properties
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        print("Starting model training...")

        history = self.model.fit(
            self.train_dataset,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_dataset,
            validation_steps=self.validation_steps
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        print("Model training completed.")
        return history


Eager execution enabled: True


In [13]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e


[2025-03-13 15:39:00,141: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-13 15:39:00,145: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-13 15:39:00,157: INFO: common: created directory at: artifacts]
[2025-03-13 15:39:00,163: INFO: common: created directory at: artifacts/training]
[2025-03-13 15:39:00,369: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model loaded and compiled.
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Data generators converted to tf.data.Datasets.
Starting model training...
23/23 ━━━━━━━━━━━━━━━━━━━━ 6s 229ms/step - accuracy: 0.6579 - loss: 1.1447 - val_accuracy: 0.9125 - val_loss: 0.1644
[2025-03-13 15:39:06,217: WARNING: saving_api: You are saving your model as an HDF5 file via `model.save()` or `keras.saving.save_model(model)`. This file format is consi